### Uso de spacy para hacer el análisis de sentimiento de los tweets.

In [1]:
# imports y configuraciones necesarias
import pandas as pd

import math

import spacy

import re
import string
import unicodedata

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

#### Cargamos los csv generados con datos en inglés y en español para probar con spacy.

Estos datos ya con sentimiento calculado para cada texto, podremos usarlos para entrenar y testear los modelos que generemos para clasificar los tweets por sentimiento.

In [2]:
# Los datos de sentimiento vienen con los valores: positivo=2, neutral=1, negativo=0
# cargamos datos en inglés
df_anotados_english = pd.read_csv('./data/df_result_english.csv', sep=',')

print("num_rows: %d\tColumnas: %d\n" % (df_anotados_english.shape[0], df_anotados_english.shape[1]) )
print("Columnas:\n", list(df_anotados_english.columns))

num_rows: 1759315	Columnas: 2

Columnas:
 ['text', 'sentiment']


In [3]:
df_anotados_english.head(10)

,text,sentiment
0,@VirginAmerica What @dhepburn said.,1.0
1,@VirginAmerica plus you've added commercials t...,2.0
2,@VirginAmerica I didn't today... Must mean I n...,1.0
3,@VirginAmerica it's really aggressive to blast...,0.0
4,@VirginAmerica and it's a really big bad thing...,0.0
5,@VirginAmerica seriously would pay $30 a fligh...,0.0
6,"@VirginAmerica yes, nearly every time I fly VX...",2.0
7,@VirginAmerica Really missed a prime opportuni...,1.0
8,"@virginamerica Well, I didn't…but NOW I DO! :-D",2.0
9,"@VirginAmerica it was amazing, and arrived an ...",2.0


Ha cargado los datos de sentimiento como float, posiblemente porque algún valor viene como NaN o similar, sustituimos esos valores por -1 y luego eliminamos esos registros antes de convertir la columna en int.

In [4]:
df_anotados_english = df_anotados_english.fillna(-1)
df_anotados_english = df_anotados_english[df_anotados_english['sentiment']!=-1]

In [5]:
c_float = df_anotados_english.columns[df_anotados_english.dtypes == float]
df_anotados_english[c_float] = df_anotados_english[c_float].astype(int)

In [6]:
df_anotados_english.head(10)

,text,sentiment
0,@VirginAmerica What @dhepburn said.,1
1,@VirginAmerica plus you've added commercials t...,2
2,@VirginAmerica I didn't today... Must mean I n...,1
3,@VirginAmerica it's really aggressive to blast...,0
4,@VirginAmerica and it's a really big bad thing...,0
5,@VirginAmerica seriously would pay $30 a fligh...,0
6,"@VirginAmerica yes, nearly every time I fly VX...",2
7,@VirginAmerica Really missed a prime opportuni...,1
8,"@virginamerica Well, I didn't…but NOW I DO! :-D",2
9,"@VirginAmerica it was amazing, and arrived an ...",2


In [7]:
# Los datos de sentimiento vienen con los valores: positivo=2, neutral=1, negativo=0
# cargamos datos en español
df_anotados_spanish = pd.read_csv('./data/df_result_spanish.csv', sep=',')

print("num_rows: %d\tColumnas: %d\n" % (df_anotados_spanish.shape[0], df_anotados_spanish.shape[1]) )
print("Columnas:\n", list(df_anotados_spanish.columns))

num_rows: 48658	Columnas: 2

Columnas:
 ['text', 'sentiment']


In [8]:
df_anotados_spanish.head(10)

,text,sentiment
0,@PauladeLasHeras No te libraras de ayudar me/n...,1
1,@marodriguezb Gracias MAR,2
2,"Off pensando en el regalito Sinde, la que se v...",0
3,Conozco a alguien q es adicto al drama! Ja ja ...,2
4,Toca @crackoviadeTV3 . Grabación dl especial N...,2
5,Buen día todos! Lo primero mandar un abrazo gr...,2
6,Desde el escaño. Todo listo para empezar #endi...,2
7,Bdías. EM no se ira de puente. Si vosotros os ...,2
8,Un sistema económico q recorta dinero para pre...,2
9,#programascambiados caca d ajuste,0


In [9]:
# vemos el recuento de los distintos valores que tiene la columna del sentimiento en el dataset
print("Recuento valores datos en inglés:",pd.value_counts(df_anotados_english['sentiment']))
print("\n")
print("Recuento valores datos en español:",pd.value_counts(df_anotados_spanish['sentiment']))

Recuento valores datos en inglés: 2    872864
0    871980
1     14470
Name: sentiment, dtype: int64


Recuento valores datos en español: 2    26204
0    19344
1     3110
Name: sentiment, dtype: int64


#### Limpieza de los textos.

Vamos a aplicar a todos los textos de los dataframes, técnicas de preprocesado y limpieza de textos para hacer más efectivos y sencillos los análisis de los textos y la clasificación del sentimiento.

In [10]:
# funciones de preprocesado y limpieza de los textos
def limpiar_tweet(tweet):
    # quitamos RT, @nombre_usuario, links y urls, hashtags, menciones, caracteres extraños o emoticonos
    tweet = re.sub('  +', ' ', tweet)
    # eliminar acentos
    tweet = ''.join((c for c in unicodedata.normalize('NFD', tweet) if unicodedata.category(c) != 'Mn'))  
    # convertir la repetición de una letra más de 2 veces a 1
    # biennnnn --> bien
    tweet = re.sub(r'(.)\1+', r'\1\1', tweet)
    # eliminar "RT", "@usuario", o los enlaces que es información que no sería útil analizar
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    tweet = re.sub('','',tweet).lower() 
    tweet = re.sub(r'http\S+', '', tweet) 
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    tweet = re.sub(r'#', '', tweet)
    tweet = re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet)
    tweet = re.sub(r'[0-9]', '', tweet) 

    return tweet 

In [11]:
# aplicamos las funciones a los textos en inglés
df_clean_english = df_anotados_english.copy()
df_clean_english['text'] = df_clean_english['text'].apply(limpiar_tweet)

In [12]:
# damos un vistazo a los textos resultantes
df_clean_english.head(20)

,text,sentiment
0,what said,1
1,plus you ve added commercials to the experie...,2
2,i didn t today must mean i need to take an...,1
3,it s really aggressive to blast obnoxious e...,0
4,and it s a really big bad thing about it,0
5,seriously would pay a flight for seats tha...,0
6,yes nearly every time i fly vx this ear wo...,2
7,really missed a prime opportunity for men wi...,1
8,well i didn t but now i do d,2
9,it was amazing and arrived an hour early y...,2


In [13]:
# aplicamos las funciones a los textos en español
df_clean_spanish = df_anotados_spanish.copy()
df_clean_spanish['text'] = df_clean_spanish['text'].apply(limpiar_tweet)

In [14]:
# damos un vistazo a los textos resultantes
df_clean_spanish.head(20)

,text,sentiment
0,no te libraras de ayudar me nos besos y gra...,1
1,gracias mar,2
2,off pensando en el regalito sinde la que se v...,0
3,conozco a alguien q es adicto al drama ja ja ...,2
4,toca grabacion dl especial navideno mari ...,2
5,buen dia todos lo primero mandar un abrazo gr...,2
6,desde el escano todo listo para empezar endia...,2
7,bdias em no se ira de puente si vosotros os ...,2
8,un sistema economico q recorta dinero para pre...,2
9,programascambiados caca d ajuste,0


#### Análisis morfosintáctico con Spacy

Cuando el texto está expresado de forma natural es normal que una palabra aparezca con distintas conjugaciones y formas, sin que cambie el significado del texto. Cuando esto ocurre un preprocesado que se realiza es convertir palabras a lemas, o eliminar categorías morfológicas que no aportan información. Se puede usar para ello la librería Spacy.

In [15]:
# spaCy utiliza modelos morfosintácticos específicos para cada idioma. Por defecto la librería no incluye ningún 
# modelo, pero podemos instalarlo de manera sencilla con comandos a python. Las siguientes líneas ejecutan un comando 
# de sistema para instalar el modelo de spaCy para el idioma inglés y español.
!python -m spacy download en
!python -m spacy download es

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/Users/josemanuel/anaconda3/lib/python3.7/site-packages/en_core_web_sm -->
/Users/josemanuel/anaconda3/lib/python3.7/site-packages/spacy/data/en
You can now load the model via spacy.load('en')
✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_sm')
✔ Linking successful
/Users/josemanuel/anaconda3/lib/python3.7/site-packages/es_core_news_sm -->
/Users/josemanuel/anaconda3/lib/python3.7/site-packages/spacy/data/es
You can now load the model via spacy.load('es')


In [16]:
# cargar modelos en memoria
nlp_eng = spacy.load('en')
nlp_spa = spacy.load('es')

In [17]:
# Ejemplo de uso de Spacy. Lo más interesante son los diferentes campos con información extra que contiene cada token
frase = "Phrase detected as an example of the use of Spacy library to analyze texts."
doc = nlp_eng(frase)

for token in doc:
    print("Token:", token)
    
pd.DataFrame(columns=["token", "lema", "POS", "tag", "shap", "isalpha", "isstop", "padre", "dep"],
    data=[[token.text, token.lemma_, token.pos_, token.tag_,
        token.shape_, token.is_alpha, token.is_stop, token.head, token.dep_]
        for token in doc]
)

Token: Phrase
Token: detected
Token: as
Token: an
Token: example
Token: of
Token: the
Token: use
Token: of
Token: Spacy
Token: library
Token: to
Token: analyze
Token: texts
Token: .


,token,lema,POS,tag,shap,isalpha,isstop,padre,dep
0,Phrase,phrase,NOUN,NN,Xxxxx,True,False,detected,nsubj
1,detected,detect,VERB,VBD,xxxx,True,False,detected,ROOT
2,as,as,ADP,IN,xx,True,True,detected,prep
3,an,an,DET,DT,xx,True,True,example,det
4,example,example,NOUN,NN,xxxx,True,False,as,pobj
5,of,of,ADP,IN,xx,True,True,example,prep
6,the,the,DET,DT,xxx,True,True,use,det
7,use,use,NOUN,NN,xxx,True,False,of,pobj
8,of,of,ADP,IN,xx,True,True,use,prep
9,Spacy,Spacy,PROPN,NNP,Xxxxx,True,False,library,compound


Visto el funcionamiento de spaCy, vamos a pasar a ejecutar el análisis morfosintáctico para cada texto de nuestros datos. Primero para los datos en inglés y luego para los datos en español.

El número de datos en inglés es muy grande, para evitar problemas por tiempo y coste computacional a la hora de realizar los procesamientos.

In [18]:
df_sample_english = df_clean_english.sample(frac=0.1, replace=False, random_state=1)

In [19]:
print("num_rows: %d\tColumnas: %d\n" % (df_sample_english.shape[0], df_sample_english.shape[1]) )
print("Columnas:\n", list(df_sample_english.columns))

num_rows: 175931	Columnas: 2

Columnas:
 ['text', 'sentiment']


In [20]:
# reparto de valores del atributo sentimiento del sample generado de los datos en inglés
print("Valores dataset:\n",pd.value_counts(df_sample_english['sentiment']))

Valores dataset:
 2    87560
0    86882
1     1489
Name: sentiment, dtype: int64


In [21]:
# utilizamos una función para ejecutar dicho análisis morfosintáctico a cada texto de los dataframes
# lo aplicamos al dataframe de datos en inglés
def add_analyzed(df):
    analyzed = [nlp_eng(text) for text in df["text"]]
    df["analyzed"] = pd.Series(analyzed, index = df.index)
    
add_analyzed(df_sample_english)
df_sample_english.head()

,text,sentiment,analyzed
819606,wants a macbook windows sucks for creating be...,0,"(wants, a, macbook, , windows, sucks, for, cr..."
1688869,music does something that is indescribable but...,2,"(music, does, something, that, is, indescribab..."
884831,williams oh it was i didn t get done until...,0,"( , williams, oh, , it, was, , i, didn, t, ..."
754246,spa felt good and now its time to sleep soo ...,0,"(spa, felt, good, , and, now, its, time, to, ..."
269213,she looks sad is it bcos her gay brother was ...,0,"(she, looks, sad, , is, it, bcos, her, gay, b..."


In [ ]:
# lo aplicamos a los datos en español
def add_analyzed_spa(df):
    analyzed = [nlp_spa(text) for text in df["text"]]
    df["analyzed"] = pd.Series(analyzed, index = df.index)
    
add_analyzed_spa(df_clean_spanish)
df_clean_spanish.head()

Vamos a sacar partido a la información morfológica que nos proporciona spaCy para mejorar el modelo predictivo. Para ello realizaremos las operaciones:

* Filtrar los textos para quedarnos solo con aquellas palabras de las categorías morfológicas con más carga de emoción.
* Filtrar los textos para no incluir palabras stopwords.
* Sustituir cada token por su lema, para así reducir el tamaño del vocabulario y simplificar el problema.

In [25]:
# utilizamos otra función para aplicar a los textos los filtrados y transformaciones comentados
# primero a los textos en inglés y luego a los textos en español igual que anteriormente
def add_posfilter(df):
    posfilter = [" ".join([token.lemma_ for token in text 
                           if token.pos_ in {"NOUN", "VERB", "ADJ", "ADV"} and not token.is_stop]) 
                 for text in df["analyzed"]]
    df["posfilter"] = pd.Series(posfilter, index = df.index)
    
add_posfilter(df_sample_english)
df_sample_english.head()

,text,sentiment,analyzed,posfilter
128038,seems i always end up at tx schl email lori...,2,"( , seems, i, always, end, up, at, tx, schl, ...",end tx schl email lori com add mail list
491756,yeah don t worry you will there s still ...,0,"( , yeah, , don, t, worry, , you, will, ,...",don t worry s week half hit road weekend s ful...
470925,excited that my email works reliably now with ...,0,"(excited, that, my, email, works, reliably, no...",excited email work reliably secondary dns back...
491264,yea that s the sad part,0,"( , yea, that, s, the, sad, part)",s sad
836490,you let me know i think you too busy for li...,0,"( , you, let, me, know, , i, think, you, too...",let know think busy lil old netta


In [26]:
add_posfilter(df_clean_spanish)
df_clean_spanish.head()

,text,sentiment,analyzed,posfilter
0,no te libraras de ayudar me nos besos y gra...,1,"( , no, te, libraras, de, ayudar, me, nos, ,...",librar ayudar beso gracia
1,gracias mar,2,"( , gracias, mar)",gracia mar
2,off pensando en el regalito sinde la que se v...,0,"(off, pensando, en, el, regalito, sinde, , la...",pensar regalito sinde sgae corrupto sacar conc...
3,conozco a alguien q es adicto al drama ja ja ...,2,"(conozco, a, alguien, q, es, adicto, al, drama...",conocer adicto drama sonar d
4,toca grabacion dl especial navideno mari ...,2,"(toca, , grabacion, dl, especial, navideno...",tocar grabacion especial navideno mari crisma


In [27]:
# rellenamos con espacios los datos vacíos por si hubiera alguno y pudiera dar problemas posteriormente
df_sample_english = df_sample_english.fillna(' ')
df_clean_spanish = df_clean_spanish.fillna(' ')

In [28]:
df_sample_english.head(10)

,text,sentiment,analyzed,posfilter
128038,seems i always end up at tx schl email lori...,2,"( , seems, i, always, end, up, at, tx, schl, ...",end tx schl email lori com add mail list
491756,yeah don t worry you will there s still ...,0,"( , yeah, , don, t, worry, , you, will, ,...",don t worry s week half hit road weekend s ful...
470925,excited that my email works reliably now with ...,0,"(excited, that, my, email, works, reliably, no...",excited email work reliably secondary dns back...
491264,yea that s the sad part,0,"( , yea, that, s, the, sad, part)",s sad
836490,you let me know i think you too busy for li...,0,"( , you, let, me, know, , i, think, you, too...",let know think busy lil old netta
371404,well susan didn t make it runner up to...,0,"( , well, , susan, didn, t, make, it, ,...",susan didn t runner bunch dancer gobsmacke s word
73350,before the cool runs out ima be trying my b...,2,"( , before, the, cool, runs, out, , i, m, a,...",cool run be try best
1166160,lmao not today sir sorry sir i did go yester...,2,"( , lmao, not, today, sir, sorry, sir, i, did...",lmao today sir sorry sir yesterday sir
1070017,loves that lubbock is wet its about time no ...,2,"(loves, that, lubbock, is, wet, , its, about,...",love lubbock wet time strip run
229521,i want a golden retriever puppy soo cute d...,0,"(i, want, a, golden, retriever, puppy, , soo...",want golden retriever puppy soo cute damn want


In [29]:
df_clean_spanish.head(10)

,text,sentiment,analyzed,posfilter
0,no te libraras de ayudar me nos besos y gra...,1,"( , no, te, libraras, de, ayudar, me, nos, ,...",librar ayudar beso gracia
1,gracias mar,2,"( , gracias, mar)",gracia mar
2,off pensando en el regalito sinde la que se v...,0,"(off, pensando, en, el, regalito, sinde, , la...",pensar regalito sinde sgae corrupto sacar conc...
3,conozco a alguien q es adicto al drama ja ja ...,2,"(conozco, a, alguien, q, es, adicto, al, drama...",conocer adicto drama sonar d
4,toca grabacion dl especial navideno mari ...,2,"(toca, , grabacion, dl, especial, navideno...",tocar grabacion especial navideno mari crisma
5,buen dia todos lo primero mandar un abrazo gr...,2,"(buen, dia, todos, , lo, primero, mandar, un,...",mandar abrazar grande miguel familia grandeza ...
6,desde el escano todo listo para empezar endia...,2,"(desde, el, escano, , todo, listo, para, empe...",escano listar empezar endiascomohoy congreso
7,bdias em no se ira de puente si vosotros os ...,2,"(bdias, , em, no, se, ira, de, puente, , si,...",bdias ira puente tableta pc orbyt decir
8,un sistema economico q recorta dinero para pre...,2,"(un, sistema, economico, q, recorta, dinero, p...",sistema economico recortar dinero prestación s...
9,programascambiados caca d ajuste,0,"(programascambiados, caca, d, ajuste)",programascambiados caca ajustar


#### Separamos los datos en training y test para los modelos

In [30]:
X_eng = df_sample_english['posfilter']
y_eng = df_sample_english['sentiment']

X_eng_train, X_eng_test, y_eng_train, y_eng_test = train_test_split(X_eng, y_eng, test_size=0.3, random_state=42)

X_spa = df_clean_spanish['posfilter']
y_spa = df_clean_spanish['sentiment']

X_spa_train, X_spa_test, y_spa_train, y_spa_test = train_test_split(X_spa, y_spa, test_size=0.3, random_state=42)

In [31]:
# montamos un modelo basado en el análisis de palabras
pipeline = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('classifier', LinearSVC())
    ]
)

params = {
    'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'vectorizer__analyzer' : ['word'],
    'vectorizer__ngram_range' : [(1, 1), (1,2), (1,3)]
}

# lo entrenamos y evaluamos con los datos en inglés
model = GridSearchCV(pipeline, params, n_jobs = 7)
model.fit(X_eng_train.values, y_eng_train)
model.score(X_eng_test.values, y_eng_test)

/Users/josemanuel/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


0.7705191360363774

In [32]:
# entrenamos y evaluamos otro modelo con los datos en español
model_spa = GridSearchCV(pipeline, params, n_jobs = 7)
model_spa.fit(X_spa_train.values, y_spa_train)
model_spa.score(X_spa_test.values, y_spa_test)

/Users/josemanuel/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


0.8068228524455405

Este problema es desbalanceado, ya que existen más datos de un sentimiento en concreto, que de otros. Por ello, usar la precisión o accuracy como métrica de la calidad del modelo no es aconsejable, ya que un modelo que clasifique todas los textos de un sentimiento en concreto podrá tener una precisión muy alta, a pesar de su falta de utilidad en práctica. En su lugar deberá emplearse la métrica roc_auc_score, que tiene en cuenta la importancia de ambas clases.

In [43]:
# eliminamos del dataframe los registros con sentimiento == 1, que serían los datos con sentimiento neutro, ya
# que esta métrica sólo debe usarse con clases que tengan dos posibles valores.
df_sample_english_copy = df_sample_english.copy()
df_clean_spanish_copy = df_clean_spanish.copy()

df_sample_english_copy = df_sample_english_copy[df_sample_english_copy.sentiment != 1]
df_clean_spanish_copy = df_clean_spanish_copy[df_clean_spanish_copy.sentiment != 1]

In [44]:
print("Valores dataset english:\n",pd.value_counts(df_sample_english_copy['sentiment']))
print("\n")
print("Valores dataset spanish:\n",pd.value_counts(df_clean_spanish_copy['sentiment']))

Valores dataset english:
 2    87553
0    86966
Name: sentiment, dtype: int64


Valores dataset spanish:
 2    26204
0    19344
Name: sentiment, dtype: int64


In [45]:
df_sample_english_copy.head(10)

,text,sentiment,analyzed,posfilter
128038,seems i always end up at tx schl email lori...,2,"( , seems, i, always, end, up, at, tx, schl, ...",end tx schl email lori com add mail list
491756,yeah don t worry you will there s still ...,0,"( , yeah, , don, t, worry, , you, will, ,...",don t worry s week half hit road weekend s ful...
470925,excited that my email works reliably now with ...,0,"(excited, that, my, email, works, reliably, no...",excited email work reliably secondary dns back...
491264,yea that s the sad part,0,"( , yea, that, s, the, sad, part)",s sad
836490,you let me know i think you too busy for li...,0,"( , you, let, me, know, , i, think, you, too...",let know think busy lil old netta
371404,well susan didn t make it runner up to...,0,"( , well, , susan, didn, t, make, it, ,...",susan didn t runner bunch dancer gobsmacke s word
73350,before the cool runs out ima be trying my b...,2,"( , before, the, cool, runs, out, , i, m, a,...",cool run be try best
1166160,lmao not today sir sorry sir i did go yester...,2,"( , lmao, not, today, sir, sorry, sir, i, did...",lmao today sir sorry sir yesterday sir
1070017,loves that lubbock is wet its about time no ...,2,"(loves, that, lubbock, is, wet, , its, about,...",love lubbock wet time strip run
229521,i want a golden retriever puppy soo cute d...,0,"(i, want, a, golden, retriever, puppy, , soo...",want golden retriever puppy soo cute damn want


In [46]:
df_clean_spanish_copy.head(10)

,text,sentiment,analyzed,posfilter
1,gracias mar,2,"( , gracias, mar)",gracia mar
2,off pensando en el regalito sinde la que se v...,0,"(off, pensando, en, el, regalito, sinde, , la...",pensar regalito sinde sgae corrupto sacar conc...
3,conozco a alguien q es adicto al drama ja ja ...,2,"(conozco, a, alguien, q, es, adicto, al, drama...",conocer adicto drama sonar d
4,toca grabacion dl especial navideno mari ...,2,"(toca, , grabacion, dl, especial, navideno...",tocar grabacion especial navideno mari crisma
5,buen dia todos lo primero mandar un abrazo gr...,2,"(buen, dia, todos, , lo, primero, mandar, un,...",mandar abrazar grande miguel familia grandeza ...
6,desde el escano todo listo para empezar endia...,2,"(desde, el, escano, , todo, listo, para, empe...",escano listar empezar endiascomohoy congreso
7,bdias em no se ira de puente si vosotros os ...,2,"(bdias, , em, no, se, ira, de, puente, , si,...",bdias ira puente tableta pc orbyt decir
8,un sistema economico q recorta dinero para pre...,2,"(un, sistema, economico, q, recorta, dinero, p...",sistema economico recortar dinero prestación s...
9,programascambiados caca d ajuste,0,"(programascambiados, caca, d, ajuste)",programascambiados caca ajustar
10,buen viernes,2,"(buen, viernes)",viernes


In [47]:
# formación de los dataframes de training y de test para sacar los datos de test a pasarle a la métrica
X_eng = df_sample_english_copy['posfilter']
y_eng = df_sample_english_copy['sentiment']

X_eng_train, X_eng_test, y_eng_train, y_eng_test = train_test_split(X_eng, y_eng, test_size=0.3, random_state=42)

X_spa = df_clean_spanish_copy['posfilter']
y_spa = df_clean_spanish_copy['sentiment']

X_spa_train, X_spa_test, y_spa_train, y_spa_test = train_test_split(X_spa, y_spa, test_size=0.3, random_state=42)

In [48]:
# lo entrenamos y evaluamos con los datos en inglés sin valores catalogados como neutros
model_eng = GridSearchCV(pipeline, params, n_jobs = 7)
model_eng.fit(X_eng_train.values, y_eng_train)
model_eng.score(X_eng_test.values, y_eng_test)

/Users/josemanuel/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


0.775097410038964

In [49]:
# evaluamos resultados del modelo con la métrica roc_auc_score con los datos en inglés
y_pred = model_eng.predict(X_eng_test)

roc_auc = roc_auc_score(y_eng_test, y_pred)

roc_auc

0.7749656890080583

In [50]:
# lo entrenamos y evaluamos con los datos en español sin valores catalogados como neutros
model_spa_noNeutro = GridSearchCV(pipeline, params, n_jobs = 7)
model_spa_noNeutro.fit(X_spa_train.values, y_spa_train)
model_spa_noNeutro.score(X_spa_test.values, y_spa_test)

/Users/josemanuel/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


0.8611781924624954

In [51]:
# evaluamos resultados del modelo con la métrica roc_auc_score con los datos en español
y_pred_spa = model_spa_noNeutro.predict(X_spa_test)

roc_auc_spa = roc_auc_score(y_spa_test, y_pred_spa)

roc_auc_spa

0.8524064451013114

#### Conclusiones del uso de Spacy

Se obtienen mejores resultados con Spacy y el modelo utilizado que con las pruebas que se hicieron con textblob, aunque muy similares a las medidas obtenidas con el clasificador Naive Bayes. 

Se mide con una métrica más fiable como "roc_auc_score". Aunque para poder evaluar el modelo con esta métrica, deben ser solo dos clases, por lo que eliminamos los datos con sentimiento neutro, y así es como se obtienen los mejores resultados. Suponemos que con menos clases es más sencillo el aprendizaje.

Se vuelve a obtener mejores resultados al entrenar y testear con los datos en español que con los datos en inglés, como ya pasaba en pruebas anteriores con el clasificador de textblob.